In [35]:
#Projeto Imersão

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

def call_agent(agent: Agent, message_text: str) -> str:

    session_service = InMemorySessionService()

    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [36]:
!pip install google-generativeai

In [27]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [32]:
!pip install -q streamlit
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [38]:
# Configurar a ferramenta de busca do Google (lembre-se de configurar sua chave de API)
##########################################
# --- Agente 1: Buscador de Receitas --- #
##########################################
def agente_buscador_receitas(ingrediente):
    buscador = Agent(
        name="agente_buscador_receitas",
        model="gemini-2.0-flash",
        description="Buscar receitas no google com base nos ingredientes fornecidos pelo usuário.",
        tools=[google_search],
        instruction="""Você é um agente especializado em encontrar receitas deliciosas na internet.
        Sua tarefa é usar a ferramenta do google (google_search) para buscar a receita mais relevante
        com base nos ingredientes principais fornecido pelo usuário, e forneça informações
        como o título da receita e um breve resumo ou os ingredientes principais, se disponíveis no resultado da busca.
        Mantenha o foco em receitas práticas e acessíveis."""
    )
    entrada_do_agente_buscador = f"Ingrediente principal: {ingrediente}"
    # Executa o agente
    receitas_encontradas = call_agent(buscador, entrada_do_agente_buscador)
    return receitas_encontradas

##################################################
# --- Agente 2: Buscador de Imagens de Receitas --- #
##################################################
def agente_buscador_imagens(nome_da_receita):
    buscador_imagens = Agent(
        name="agente_buscador_imagens",
        model="gemini-2.0-flash",
        description="Buscar imagens correspondentes a uma receita específica no google.",
        tools=[google_search],
        instruction="""Você é um especialista em encontrar imagens online.
        Sua tarefa é usar a ferramenta do google (google_search) para buscar uma imagem que represente bem a receita fornecida pelo agente 1.
        A imagem deve ser visualmente atraente e relevante para o nome da receita.
        Retorne apenas a URL da imagem encontrada."""
    )
    entrada_do_agente_buscador_imagens = f"Nome da receita: {nome_da_receita}"
    # Executa o agente
    url_da_imagem = call_agent(buscador_imagens, entrada_do_agente_buscador_imagens)
    return url_da_imagem

######################################################
# --- Agente 3: Criador de Post de Receita para Site --- #
######################################################
def agente_criador_post(receitas_encontradas):
    criador_post = Agent(
        name="agente_criador_post",
        model="gemini-2.0-flash",
        description="Juntar informações de receitas e imagens para criar um post para site.",
        instruction="""Você é um criador de conteúdo para um site de receitas.
        Com base nas informações da receita encontrada, sua tarefa é criar um post bem estruturado.
        Inclua o título, os ingredientes (se disponíveis no resumo da busca),
        um breve modo de preparo (se disponíveis no resumo), e dicas adicionais para o leitor.
        Se você tiver a URL de uma imagem correspondente, mencione que uma imagem pode ser adicionada ao post.
        Organize o post de forma clara e convidativa para os leitores do site."""
    )
    entrada_do_agente_criador_post = f"Informações das receitas: {receitas_encontradas}"
    # Executa o agente
    post_para_site = call_agent(criador_post, entrada_do_agente_criador_post)
    return post_para_site

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🍳 Iniciando o Sistema de Criação de Posts de Receitas 🍳")

# --- Obter o Ingrediente do Usuário ---
ingrediente_principal = input("🍎 Por favor, digite o ingrediente principal para buscar receitas: ")

# Inserir lógica do sistema de agentes para receitas ################################################

if not ingrediente_principal:
    print("Você não digitou um ingrediente.")
else:
    print(f"Ótima escolha! Vamos buscar receitas com: {ingrediente_principal}")

    receitas_buscadas = agente_buscador_receitas(ingrediente_principal)

    # Vamos pegar o título da primeira receita encontrada para buscar uma imagem
    if receitas_buscadas and isinstance(receitas_buscadas, str):
        # Tenta extrair o título da primeira receita (pode precisar de ajustes dependendo do formato da saída)
        primeira_receita = receitas_buscadas.split('\n')[0].replace("Título: ", "").strip() if "Título:" in receitas_buscadas.split('\n')[0] else ingrediente_principal
        url_imagem = agente_buscador_imagens(primeira_receita)
        print(f"Foto da receita: {url_imagem}")
        post_final = agente_criador_post(receitas_buscadas)
        print("\n--- Resultado ---\n")
        display(to_markdown(post_final + f"\n\n**Imagem:** {url_imagem}"))
        print("-----------------------------------------------------")
    else:
        print("Nenhuma receita foi encontrada. Não foi possível buscar uma imagem ou criar o post.")

🍳 Iniciando o Sistema de Criação de Posts de Receitas 🍳
🍎 Por favor, digite o ingrediente principal para buscar receitas: farinha
Ótima escolha! Vamos buscar receitas com: farinha
Foto da receita: Não consigo fornecer uma URL de imagem específica. No entanto, posso sugerir alguns links de onde você pode encontrar imagens de farinha:

*   **Freepik:** Oferece diversas fotos gratuitas de farinha de trigo, incluindo fotos com espigas de trigo e ingredientes para bolos.
*   **Vecteezy:** Possui uma vasta coleção de imagens e fotos de farinha, com opções gratuitas e pagas.
*   **Unsplash:** Apresenta uma variedade de imagens de farinha de trigo de alta qualidade, gratuitas para uso comercial.
*   **iStock:** Dispõe de um extenso banco de imagens de farinha, com fotos royalty-free.


--- Resultado ---



> ## Delícias da Cozinha: Receitas Criativas com Farinha para Todos os Gostos!
> 
> A farinha é um ingrediente versátil e essencial em qualquer cozinha, capaz de transformar preparações simples em pratos incrivelmente saborosos. Seja para um lanche rápido, um prato principal reconfortante ou uma sobremesa irresistível, a farinha é a base de inúmeras receitas que aquecem o coração e adoçam o paladar.
> 
> **Opções Salgadas para Saborear:**
> 
> Se você é fã de pratos salgados, prepare-se para explorar uma variedade de opções que vão te surpreender. Que tal começar com um delicioso camarão ao molho branco, perfeito para um jantar especial? Ou, quem sabe, uma torta-pão de presunto e queijo, ideal para um lanche reforçado?
> 
> Para os amantes de empadão, experimente a versão com massa de grão de bico, uma opção nutritiva e saborosa. E que tal um fricassê de carne moída, prático e delicioso para o dia a dia? Para finalizar, o pastel assado com três queijos é uma excelente pedida para compartilhar com amigos e familiares.
> 
> **Doce Tentação: Receitas Açucaradas com Farinha:**
> 
> Se a sua paixão é por doces, prepare-se para se deliciar com as opções que a farinha pode proporcionar. Comece com um brownie super fácil, perfeito para matar aquela vontade de chocolate. Para os fãs de frutas cítricas, a tortinha de limão com brigadeiro é uma combinação irresistível.
> 
> **Receita Coringa: Farinha, Água e Sal:**
> 
> E para aqueles momentos em que a simplicidade é a chave, que tal experimentar uma receita que leva apenas três ingredientes: farinha, água e sal? Com essa combinação básica, é possível criar pães, massas e outras preparações deliciosas e econômicas.
> 
> **Dicas Extras:**
> 
> *   **Experimente diferentes tipos de farinha:** Além da tradicional farinha de trigo, explore outras opções como farinha de arroz, farinha de aveia, farinha de amêndoas e farinha de coco para adicionar novos sabores e texturas às suas receitas.
> *   **Atenção às medidas:** Para garantir o sucesso de suas receitas, siga as medidas dos ingredientes com precisão. Utilize xícaras e colheres medidoras para obter os melhores resultados.
> *   **Solte a criatividade:** Não tenha medo de experimentar e adaptar as receitas de acordo com suas preferências e ingredientes disponíveis. A cozinha é um espaço para a criação e a experimentação!
> 
> **[Espaço para adicionar uma imagem de um prato feito com farinha]**
> 
> Agora é só escolher sua receita favorita, colocar a mão na massa e desfrutar de momentos deliciosos na cozinha!
> 
> 
> **Imagem:** Não consigo fornecer uma URL de imagem específica. No entanto, posso sugerir alguns links de onde você pode encontrar imagens de farinha:
> 
> *   **Freepik:** Oferece diversas fotos gratuitas de farinha de trigo, incluindo fotos com espigas de trigo e ingredientes para bolos.
> *   **Vecteezy:** Possui uma vasta coleção de imagens e fotos de farinha, com opções gratuitas e pagas.
> *   **Unsplash:** Apresenta uma variedade de imagens de farinha de trigo de alta qualidade, gratuitas para uso comercial.
> *   **iStock:** Dispõe de um extenso banco de imagens de farinha, com fotos royalty-free.


-----------------------------------------------------
